<a href="https://colab.research.google.com/github/manigamer22/weatherwise-subash-kovvuri/blob/main/starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌦️ WeatherWise – Starter Notebook

Welcome to your **WeatherWise** project notebook! This scaffold is designed to help you build your weather advisor app using Python, visualisations, and AI-enhanced development.

---

📄 **Full Assignment Specification**  
See [`ASSIGNMENT.md`](ASSIGNMENT.md) or check the LMS for full details.

📝 **Quick Refresher**  
A one-page summary is available in [`resources/assignment-summary.md`](resources/assignment-summary.md).

---

🧠 **This Notebook Structure is Optional**  
You’re encouraged to reorganise, rename sections, or remove scaffold cells if you prefer — as long as your final version meets the requirements.

✅ You may delete this note before submission.




## 🧰 Setup and Imports

This section imports commonly used packages and installs any additional tools used in the project.

- You may not need all of these unless you're using specific features (e.g. visualisations, advanced prompting).
- The notebook assumes the following packages are **pre-installed** in the provided environment or installable via pip:
  - `requests`, `matplotlib`, `pyinputplus`
  - `fetch-my-weather` (for accessing weather data easily)
  - `hands-on-ai` (for AI logging, comparisons, or prompting tools)

If you're running this notebook in **Google Colab**, uncomment the following lines to install the required packages.


In [ ]:
# 🧪 Optional packages — uncomment if needed in Colab or JupyterHub
!pip install fetch-my-weather
!pip install hands-on-ai
!pip install pyinputplus

# ✅ Import after installing (if needed)
from fetch_my_weather import get_weather
from hands_on_ai import prompt_logger

#api: key: dc54ae66a5dcb61c0e4816e5be860da2   <-- use this key to get all the data for the weather!

# Go to https://openweathermap.org/api to got my own API key.

import os

os.environ['HANDS_ON_AI_SERVER'] = "https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={API key}"
os.environ['HANDS_ON_AI_MODEL'] = 'granite3.2'
os.environ['HANDS_ON_AI_API_KEY'] = input('Enter your API key: ')


## 📦 Setup and Configuration
Import required packages and setup environment.

In [ ]:
# ✅ STEP 1: Install Required Packages
!pip install fetch-my-weather --quiet
!pip install hands-on-ai --quiet
!pip install pyinputplus --quiet
!pip install matplotlib --quiet

# ✅ STEP 2: Imports
import requests
import matplotlib.pyplot as plt
import pyinputplus as pyip
from fetch_my_weather import get_weather
from hands_on_ai.chat import get_response

## 🌤️ Weather Data Functions

In [ ]:
API_KEY = 'dc54ae66a5dcb61c0e4816e5be860da2'

# ✅ STEP 4: Weather Data Fetching Function
def get_weather_data(city):
    """
    Fetches current weather data for the specified city from OpenWeatherMap API.

    Parameters:
        city (str): The name of the city to fetch weather for.

    Returns:
        dict or None: JSON response containing weather data if successful, else None.
    """
    # Construct the API request URL with city, API key, and metric units
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric"
    try:
        response = requests.get(url)
        # Check if the request was successful
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error: Unable to fetch weather data ({response.status_code})")
            return None
    except requests.exceptions.RequestException as e:
        # Handle exceptions like connection errors
        print(f"Request failed: {e}")
        return None

## 📊 Visualisation Functions

In [ ]:
def show_temperature_chart(temps, days):
    """
    Creates a 3D-style colorful temperature line chart showing temperature trends over days.

    Parameters:
        temps (list of int/float): Temperature values to plot.
        days (list of str): Corresponding days for the temperature values.
    """
    # Use seaborn-dark style for better visualization aesthetics
    plt.style.use('seaborn-dark')
    plt.figure(figsize=(10, 6))
    # Plot line chart with orange color and markers for each day
    plt.plot(days, temps, marker='o', linestyle='-', linewidth=2, color='orange')
    # Fill the area under the line with semi-transparent orange color
    plt.fill_between(days, temps, color='orange', alpha=0.3)
    plt.title("Temperature Forecast", fontsize=16)
    plt.xlabel("Day", fontsize=12)
    plt.ylabel("Temperature (°C)", fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.show()


def show_precipitation_chart(precip, times):
    """
    Creates a bar chart for precipitation percentages over time with a dark theme.

    Parameters:
        precip (list of int/float): Precipitation values (percentage).
        times (list of str): Corresponding times or labels for the precipitation values.
    """
    plt.style.use('dark_background')
    plt.figure(figsize=(10, 6))
    # Bar chart with dodgerblue bars and white edges for contrast
    bars = plt.bar(times, precip, color='dodgerblue', edgecolor='white')
    plt.title("Precipitation Forecast", fontsize=16)
    plt.xlabel("Time", fontsize=12)
    plt.ylabel("Precipitation (%)", fontsize=12)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.grid(True, linestyle='--', alpha=0.3)
    plt.show()

## 🤖 Natural Language Processing

In [ ]:
def parse_weather_question(query):
    """
    Parses the user's weather-related question and identifies the type of intent.

    Parameters:
        query (str): The user input question.

    Returns:
        str: The identified query type ('umbrella', 'temperature', 'rain', or 'unknown').
    """
    query = query.lower()
    # Check keywords to determine user intent
    if "umbrella" in query:
        return "umbrella"
    elif "temperature" in query:
        return "temperature"
    elif "rain" in query:
        return "rain"
    return "unknown"

def generate_weather_response(query_type, weather_data):
    """
    Generates a natural language response based on the query type and weather data.

    Parameters:
        query_type (str): The type of question parsed.
        weather_data (dict): JSON weather data from API.

    Returns:
        str: A response string tailored to the user's question.
    """
    if query_type == "umbrella":
        # Check weather condition for rain to decide umbrella need
        condition = weather_data["weather"][0]["main"].lower()
        return "Yes, bring an umbrella." if "rain" in condition else "No umbrella needed."
    elif query_type == "temperature":
        # Provide current temperature
        temp = weather_data["main"]["temp"]
        return f"The current temperature is {temp}°C."
    elif query_type == "rain":
        # Describe rain or other precipitation
        condition = weather_data["weather"][0]["description"]
        return f"Weather forecast: {condition}."
    else:
        return "I didn't understand that question."

## 🧭 User Interface

In [8]:
def menu():
    """
    Main interactive menu loop for the WeatherWise application.
    Allows user to fetch weather, ask questions, view charts, or exit.
    """
    while True:
        # Display a numbered menu for user selection
        choice = pyip.inputMenu(['Get Weather', 'Ask a Question', 'Show Chart', 'Exit'], numbered=True)

        if choice == 'Get Weather':
            # Prompt user for city and fetch current weather data
            city = input("Enter city: ")
            data = get_weather_data(city)
            if data:
                # Display summary of weather and temperature
                print(f"{city.title()} Weather: {data['weather'][0]['description']}, Temp: {data['main']['temp']}°C")
            else:
                print("City not found or API error.")

        elif choice == 'Ask a Question':
            # Allow user to ask natural language questions about weather
            city = input("Enter city: ")
            data = get_weather_data(city)
            if not data:
                print("City not found.")
                continue
            query = input("Ask a weather question (e.g., Do I need an umbrella?): ")
            # Parse question intent and generate appropriate response
            q_type = parse_weather_question(query)
            response = generate_weather_response(q_type, data)
            print(response)

        elif choice == 'Show Chart':
            # Show a sample temperature forecast chart (placeholder data)
            days = ["Mon", "Tue", "Wed", "Thu", "Fri"]
            temps = [21, 23, 22, 24, 25]  # TODO: Replace with real forecast data if available
            show_temperature_chart(temps, days)
        else:
            # Exit the program loop
            print("Goodbye!")
            break

## 🧩 Main Application Logic

In [ ]:
# ✅ STEP 1: Install Required Packages
# Installing external libraries needed for weather fetching, AI chat, user input validation, and plotting
!pip install fetch-my-weather --quiet
!pip install hands-on-ai --quiet
!pip install pyinputplus --quiet
!pip install matplotlib --quiet

# ✅ STEP 2: Imports
# Importing necessary modules for HTTP requests, plotting, user input validation, and AI chat interaction
import requests
import matplotlib.pyplot as plt
import pyinputplus as pyip
from fetch_my_weather import get_weather
from hands_on_ai.chat import get_response

# ✅ STEP 3: Weather API Setup
# OpenWeatherMap API key for authentication with the weather data service
API_KEY = 'dc54ae66a5dcb61c0e4816e5be860da2'

# ✅ STEP 4: Weather Data Fetching Function
def get_weather_data(city):
    """
    Fetches current weather data for the specified city from OpenWeatherMap API.

    Parameters:
        city (str): The name of the city to fetch weather for.

    Returns:
        dict or None: JSON response containing weather data if successful, else None.
    """
    # Construct the API request URL with city, API key, and metric units
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric"
    try:
        response = requests.get(url)
        # Check if the request was successful
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error: Unable to fetch weather data ({response.status_code})")
            return None
    except requests.exceptions.RequestException as e:
        # Handle exceptions like connection errors
        print(f"Request failed: {e}")
        return None

# ✅ STEP 5: Visualisation Functions

def show_temperature_chart(temps, days):
    """
    Creates a 3D-style colorful temperature line chart showing temperature trends over days.

    Parameters:
        temps (list of int/float): Temperature values to plot.
        days (list of str): Corresponding days for the temperature values.
    """
    # Use seaborn-dark style for better visualization aesthetics
    plt.style.use('seaborn-dark')
    plt.figure(figsize=(10, 6))
    # Plot line chart with orange color and markers for each day
    plt.plot(days, temps, marker='o', linestyle='-', linewidth=2, color='orange')
    # Fill the area under the line with semi-transparent orange color
    plt.fill_between(days, temps, color='orange', alpha=0.3)
    plt.title("Temperature Forecast", fontsize=16)
    plt.xlabel("Day", fontsize=12)
    plt.ylabel("Temperature (°C)", fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.show()


def show_precipitation_chart(precip, times):
    """
    Creates a bar chart for precipitation percentages over time with a dark theme.

    Parameters:
        precip (list of int/float): Precipitation values (percentage).
        times (list of str): Corresponding times or labels for the precipitation values.
    """
    plt.style.use('dark_background')
    plt.figure(figsize=(10, 6))
    # Bar chart with dodgerblue bars and white edges for contrast
    bars = plt.bar(times, precip, color='dodgerblue', edgecolor='white')
    plt.title("Precipitation Forecast", fontsize=16)
    plt.xlabel("Time", fontsize=12)
    plt.ylabel("Precipitation (%)", fontsize=12)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.grid(True, linestyle='--', alpha=0.3)
    plt.show()

# ✅ STEP 6: Natural Language Processing Functions

def parse_weather_question(query):
    """
    Parses the user's weather-related question and identifies the type of intent.

    Parameters:
        query (str): The user input question.

    Returns:
        str: The identified query type ('umbrella', 'temperature', 'rain', or 'unknown').
    """
    query = query.lower()
    # Check keywords to determine user intent
    if "umbrella" in query:
        return "umbrella"
    elif "temperature" in query:
        return "temperature"
    elif "rain" in query:
        return "rain"
    return "unknown"

def generate_weather_response(query_type, weather_data):
    """
    Generates a natural language response based on the query type and weather data.

    Parameters:
        query_type (str): The type of question parsed.
        weather_data (dict): JSON weather data from API.

    Returns:
        str: A response string tailored to the user's question.
    """
    if query_type == "umbrella":
        # Check weather condition for rain to decide umbrella need
        condition = weather_data["weather"][0]["main"].lower()
        return "Yes, bring an umbrella." if "rain" in condition else "No umbrella needed."
    elif query_type == "temperature":
        # Provide current temperature
        temp = weather_data["main"]["temp"]
        return f"The current temperature is {temp}°C."
    elif query_type == "rain":
        # Describe rain or other precipitation
        condition = weather_data["weather"][0]["description"]
        return f"Weather forecast: {condition}."
    else:
        return "I didn't understand that question."

# ✅ STEP 7: User Interface Menu

def menu():
    """
    Main interactive menu loop for the WeatherWise application.
    Allows user to fetch weather, ask questions, view charts, or exit.
    """
    while True:
        # Display a numbered menu for user selection
        choice = pyip.inputMenu(['Get Weather', 'Ask a Question', 'Show Chart', 'Exit'], numbered=True)

        if choice == 'Get Weather':
            # Prompt user for city and fetch current weather data
            city = input("Enter city: ")
            data = get_weather_data(city)
            if data:
                # Display summary of weather and temperature
                print(f"{city.title()} Weather: {data['weather'][0]['description']}, Temp: {data['main']['temp']}°C")
            else:
                print("City not found or API error.")

        elif choice == 'Ask a Question':
            # Allow user to ask natural language questions about weather
            city = input("Enter city: ")
            data = get_weather_data(city)
            if not data:
                print("City not found.")
                continue
            query = input("Ask a weather question (e.g., Do I need an umbrella?): ")
            # Parse question intent and generate appropriate response
            q_type = parse_weather_question(query)
            response = generate_weather_response(q_type, data)
            print(response)

        elif choice == 'Show Chart':
            # Show a sample temperature forecast chart (placeholder data)
            days = ["Mon", "Tue", "Wed", "Thu", "Fri"]
            temps = [21, 23, 22, 24, 25]  # TODO: Replace with real forecast data if available
            show_temperature_chart(temps, days)
        else:
            # Exit the program loop
            print("Goodbye!")
            break

# ✅ STEP 8: Run the Application
# Start the interactive WeatherWise menu application
menu()


## 🧪 Testing and Examples

In [ ]:
# ✅ STEP 1: Install Required Packages
# Installing external libraries needed for weather fetching, AI chat, user input validation, and plotting
!pip install fetch-my-weather --quiet
!pip install hands-on-ai --quiet
!pip install pyinputplus --quiet
!pip install matplotlib --quiet

# ✅ STEP 2: Imports
# Importing necessary modules for HTTP requests, plotting, user input validation, and AI chat interaction
import requests
import matplotlib.pyplot as plt
import pyinputplus as pyip
from fetch_my_weather import get_weather
from hands_on_ai.chat import get_response

# ✅ STEP 3: Weather API Setup
# OpenWeatherMap API key for authentication with the weather data service
API_KEY = 'dc54ae66a5dcb61c0e4816e5be860da2'

# ✅ STEP 4: Weather Data Fetching Function
def get_weather_data(city):
    """
    Fetches current weather data for the specified city from OpenWeatherMap API.

    Parameters:
        city (str): The name of the city to fetch weather for.

    Returns:
        dict or None: JSON response containing weather data if successful, else None.
    """
    # Construct the API request URL with city, API key, and metric units
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric"
    try:
        response = requests.get(url)
        # Check if the request was successful
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error: Unable to fetch weather data ({response.status_code})")
            return None
    except requests.exceptions.RequestException as e:
        # Handle exceptions like connection errors
        print(f"Request failed: {e}")
        return None

# ✅ STEP 5: Visualisation Functions

def show_temperature_chart(temps, days):
    """
    Creates a 3D-style colorful temperature line chart showing temperature trends over days.

    Parameters:
        temps (list of int/float): Temperature values to plot.
        days (list of str): Corresponding days for the temperature values.
    """
    # Use seaborn-dark style for better visualization aesthetics
    plt.style.use('seaborn-dark')
    plt.figure(figsize=(10, 6))
    # Plot line chart with orange color and markers for each day
    plt.plot(days, temps, marker='o', linestyle='-', linewidth=2, color='orange')
    # Fill the area under the line with semi-transparent orange color
    plt.fill_between(days, temps, color='orange', alpha=0.3)
    plt.title("Temperature Forecast", fontsize=16)
    plt.xlabel("Day", fontsize=12)
    plt.ylabel("Temperature (°C)", fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.show()


def show_precipitation_chart(precip, times):
    """
    Creates a bar chart for precipitation percentages over time with a dark theme.

    Parameters:
        precip (list of int/float): Precipitation values (percentage).
        times (list of str): Corresponding times or labels for the precipitation values.
    """
    plt.style.use('dark_background')
    plt.figure(figsize=(10, 6))
    # Bar chart with dodgerblue bars and white edges for contrast
    bars = plt.bar(times, precip, color='dodgerblue', edgecolor='white')
    plt.title("Precipitation Forecast", fontsize=16)
    plt.xlabel("Time", fontsize=12)
    plt.ylabel("Precipitation (%)", fontsize=12)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.grid(True, linestyle='--', alpha=0.3)
    plt.show()

# ✅ STEP 6: Natural Language Processing Functions

def parse_weather_question(query):
    """
    Parses the user's weather-related question and identifies the type of intent.

    Parameters:
        query (str): The user input question.

    Returns:
        str: The identified query type ('umbrella', 'temperature', 'rain', or 'unknown').
    """
    query = query.lower()
    # Check keywords to determine user intent
    if "umbrella" in query:
        return "umbrella"
    elif "temperature" in query:
        return "temperature"
    elif "rain" in query:
        return "rain"
    return "unknown"

def generate_weather_response(query_type, weather_data):
    """
    Generates a natural language response based on the query type and weather data.

    Parameters:
        query_type (str): The type of question parsed.
        weather_data (dict): JSON weather data from API.

    Returns:
        str: A response string tailored to the user's question.
    """
    if query_type == "umbrella":
        # Check weather condition for rain to decide umbrella need
        condition = weather_data["weather"][0]["main"].lower()
        return "Yes, bring an umbrella." if "rain" in condition else "No umbrella needed."
    elif query_type == "temperature":
        # Provide current temperature
        temp = weather_data["main"]["temp"]
        return f"The current temperature is {temp}°C."
    elif query_type == "rain":
        # Describe rain or other precipitation
        condition = weather_data["weather"][0]["description"]
        return f"Weather forecast: {condition}."
    else:
        return "I didn't understand that question."

# ✅ STEP 7: User Interface Menu

def menu():
    """
    Main interactive menu loop for the WeatherWise application.
    Allows user to fetch weather, ask questions, view charts, or exit.
    """
    while True:
        # Display a numbered menu for user selection
        choice = pyip.inputMenu(['Get Weather', 'Ask a Question', 'Show Chart', 'Exit'], numbered=True)

        if choice == 'Get Weather':
            # Prompt user for city and fetch current weather data
            city = input("Enter city: ")
            data = get_weather_data(city)
            if data:
                # Display summary of weather and temperature
                print(f"{city.title()} Weather: {data['weather'][0]['description']}, Temp: {data['main']['temp']}°C")
            else:
                print("City not found or API error.")

        elif choice == 'Ask a Question':
            # Allow user to ask natural language questions about weather
            city = input("Enter city: ")
            data = get_weather_data(city)
            if not data:
                print("City not found.")
                continue
            query = input("Ask a weather question (e.g., Do I need an umbrella?): ")
            # Parse question intent and generate appropriate response
            q_type = parse_weather_question(query)
            response = generate_weather_response(q_type, data)
            print(response)

        elif choice == 'Show Chart':
            # Show a sample temperature forecast chart (placeholder data)
            days = ["Mon", "Tue", "Wed", "Thu", "Fri"]
            temps = [21, 23, 22, 24, 25]  # TODO: Replace with real forecast data if available
            show_temperature_chart(temps, days)
        else:
            # Exit the program loop
            print("Goodbye!")
            break

# ✅ STEP 8: Run the Application
# Start the interactive WeatherWise menu application
menu()


## 🗂️ AI Prompting Log (Optional)
Add markdown cells here summarising prompts used or link to AI conversations in the `ai-conversations/` folder.

In [ ]:
Conversation Log 1: Setting Up Weather API Integration
User:
How do I fetch weather data from OpenWeatherMap using Python?

AI:
You can use the requests library to send a GET request to their API endpoint. You’ll need your API key and the city name. Here’s a basic example:

import requests

API_KEY = 'your_api_key'
city = 'London'
url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric"

response = requests.get(url)
if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print("Failed to get data")
Make sure to replace 'your_api_key' with your actual key.

Conversation Log 2: Adding Natural Language Processing
User:
How can I make my program understand if the user is asking about rain or temperature?

AI:
A simple way is to look for keywords in the user’s input. For example:

def parse_weather_question(query):
    query = query.lower()
    if 'umbrella' in query:
        return 'umbrella'
    elif 'temperature' in query:
        return 'temperature'
    elif 'rain' in query:
        return 'rain'
    return 'unknown'
You can then use this intent to give tailored answers.

Conversation Log 3: Improving Visualizations
User:
How can I create a temperature line chart in Python with matplotlib?

AI:
Use matplotlib’s plot() function. Here’s a sample:

import matplotlib.pyplot as plt

days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri']
temps = [21, 23, 22, 24, 25]

plt.plot(days, temps, marker='o', color='orange')
plt.title('Temperature Forecast')
plt.xlabel('Day')
plt.ylabel('Temperature (°C)')
plt.grid(True)
plt.show()
This creates a simple, clear line chart.

Conversation Log 4: Debugging API Errors
User:
I keep getting a 404 error when fetching weather data. What should I check?

AI:
A 404 error usually means the requested city name might be incorrect or misspelled. Make sure:

The city name is spelled correctly.
The API endpoint URL is formatted correctly.
Your API key is valid and not expired.
You can also print the full URL before the request to debug.

Conversation Log 5: Reflection and Improvements
User:
What features could I add to improve my WeatherWise app?

AI:
Here are some ideas:

Integrate multi-day forecast data for extended visualization.
Use more advanced NLP or AI models to handle varied user queries better.
Add a graphical user interface for easier interaction.
Implement voice input or output for accessibility.
Provide weather alerts or notifications.
These features would make your app more user-friendly and functional.